In [1]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25 python-dotenv
! pip install langchain-chroma
! pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [28]:
import os
import getpass

upstage_api_key = getpass.getpass("Enter your Upstage API key: ")
os.environ.setdefault('UPSTAGE_API_KEY', upstage_api_key)

In [20]:
from langchain_upstage import UpstageLayoutAnalysisLoader


layzer = UpstageLayoutAnalysisLoader("pilot_JEJU_ENG.pdf", output_type="html")
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

We take the output of LLM 1 which is shown below and pass it to LLM 2

In [21]:
## LLM 1 Output
questions = [
    {'question': 'What is the main aim of the Jeju government in terms of Jeju culture?',
     'options': ['a. To promote modernization', 'b. To preserve traditional culture', 'c. To merge indigenous and new cultures', 'd. To focus on tangible heritages'],
     'right_option': 'b. To preserve traditional culture'
    },
    {'question': 'What is the biggest challenge in the Culture Pilot Cities programme?',
     'options': ['a. Determining the scope of the project', 'b. Establishing good practices relevant to Agenda 21', 'c. Understanding Jeju culture from a short visit', 'd. Predicting the difference between viewpoints'],
     'right_option': 'a. Determining the scope of the project'
    }
]


In [22]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)

In [24]:
from langchain_chroma import Chroma

# 3. Embed & indexing
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
)

In [25]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context.
    In the answer, just specify the letter corresponding to the option.
    If you don't know the answer, just provide your best guess and do not provide any reasoning.

    For example, if you think the answer is the first option, just write "a".
    If you think the answer is the second option, just write "b", and so on.

    ---
    Question: {question}
    ---
    Options: {options}
    ---
    Context: {context}


    chosen option (a, b, c, or d):

    """
)
chain = prompt_template | llm | StrOutputParser()

In [26]:
retriever = vectorstore.as_retriever()
result_docs = retriever.invoke(questions[0]["question"])
# Answer from LLM 2
chain.invoke({"question": questions[0]["question"], "options":questions[0]["options"] ,"context": result_docs})


'b. To preserve traditional culture'

In [27]:
# Answer from LLM 1
questions[0]['right_option']

'b. To preserve traditional culture'

### Both LLM 1 and LLM 2 are giving the same answer as output, hence the question is marked as a valid question